In [1]:
import csv
import json
import os
import glob
import re

In [3]:
filesys_path  = '/home/mike/work/scores_monthly_details/files/NM_Details_Mai_2020/'
file_mask  = 'azmedien_gluenz-visitors-overview (*).csv'
file_json_out = 'NM_Details_202005.json'
file_csv_out  = 'NM_Details_202005.csv'

In [4]:
os.chdir(filesys_path)
filelist = glob.glob(file_mask)

In [5]:
def clean_number(nstr):
    if re.match("[0-9]+\.",nstr):
        return int(nstr.replace('.',''))
    if re.match("(?:\s+)?([0-9]+),",nstr):
        return float("%s.%s" % (re.findall("\s?([0-9]+),([0-9]+)",nstr)[0][0],re.findall("\s?([0-9]+),([0-9]+)",nstr)[0][1]))
    return int(nstr)

In [6]:
def initialize():
    vars = {
    "portals" : ["AAZ","BAT","BLZ","BZB","GTB","LIZ","OT","SOZ"],
    "current_portal" : '',
    "current_section" : '',
    "current_key" : '',
    "matrix" : {}
    }
    return vars


In [7]:
# Report: Besucher/Übersicht,,,,,
# Site: aznetz,,,,,
# Zeitraum: 01.04.2020 00:00:00 GMT - 01.05.2020 00:00:00 GMT,,,,,
# Filter: /live/bat2016,,,,,
# Darstellung: Darstellung pro Tag,,,,,
# Report erstellt durch: azmedien_gluenz (azmedien_gluenz),,,,,
# Report erstellt am: 01.05.2020 05:46:43 GMT,,,,,
# Zeitraum,Pis,Visits,Pis/Visit,Viewtime,Visits Duration
pattern_1 = re.compile("^Zeitraum: [0-3][0-9].([01][0-9]).(20[0-2][0-9]) 00:00:00 GMT - [0-3][0-9].[01][0-9].20[0-2][0-9] 00:00:00 GMT")
pattern_2 = re.compile('^"?Filter: /(?:apps|nwn)/(?:apps|nwn)/([a-z][a-z][a-z]?).*(live|epaper)/\\1')
pattern_3 = re.compile('^"?Filter: /live/([a-z][a-z][a-z]?)2016.*')
pattern_4 = re.compile('^[?]')

In [8]:
vars = initialize()
for file_data_in in filelist:
    #print(file_data_in)
    sums_total = ['nil',0,0,0,0,0]
    infile = filesys_path+file_data_in
    #print('Reading ',infile)
    with open(infile,'r',encoding="iso-8859-15") as csvfile:
        vars["current_key"] = 'unknown' # Defaultwert für Fälle, die beim Matching der Filter durch die Maschen rutschen
        reader = csv.reader(csvfile, delimiter=',', quotechar='"')
        for row in reader:
            #try:
                row[0]
                sums_total[0]=row[0]
                if pattern_1.match(row[0]):
                    pass
                if row[0] == 'Filter: /nwn/apps/epaper/saw':  # entspricht nicht dem gängigen Muster
                    vars["current_key"] = ("%s %s" % ('SAW','ALL'))
                if row[0] == 'Filter: /live/saw':             # entspricht nicht dem gängigen Muster
                    vars["current_key"] = ("%s %s" % ('SAW','LIVE'))
                if pattern_2.match(row[0]):
                    vars["current_key"] = ("%s APP-%s" % (re.findall(pattern_2,row[0])[0][0],re.findall(pattern_2,row[0])[0][1])).upper()
                    #print(vars["current_key"])
                if pattern_3.match(row[0]):
                    vars["current_key"] = ("%s %s" % (re.findall(pattern_3,row[0])[0],'web')).upper()
                    #print(vars["current_key"])
                if pattern_4.match(row[0]):
                    for i in range(1,6):
                        if clean_number(row[i]) > sums_total[i]:
                            sums_total[i] = clean_number(row[i])
            #except:
            #    pass
        #print('Current Key: ',vars["current_key"])
        vars["matrix"][vars["current_key"]] = sums_total[1:]
print(vars["matrix"])

{'BAT APP-LIVE': [126155, 36839, 105.97, 3324.73, 11344.54], 'GTB WEB': [351437, 84256, 129.65, 2041.19, 8422.38], 'BAT WEB': [310238, 81468, 118.06, 2545.2, 9634.07], 'SOZ APP-EPAPER': [1611420, 86158, 593.24, 1063.58, 19945.7], 'BZB APP-LIVE': [349379, 119341, 90.64, 3843.07, 11228.53], 'BZB WEB': [3789583, 1050418, 111.76, 2647.26, 9413.2], 'OT APP-LIVE': [425794, 141325, 93.28, 3195.25, 9616.56], 'SOZ WEB': [2049109, 523246, 120.58, 2299.4, 8828.44], 'SAW LIVE': [101237, 36685, 61.45, 5083.78, 7231.69], 'BLZ WEB': [93, 37, 51.72, 400.73, 1205.36], 'SAW APP-EPAPER': [123656, 8703, 375.64, 874.18, 10501.44], 'GTB APP-EPAPER': [245616, 11787, 663.72, 1075.8, 22459.09], 'AAZ WEB': [12583297, 3313170, 117.62, 2238.95, 8404.48], 'OT WEB': [698138, 189063, 114.51, 2792.79, 10187.51], 'GTB APP-LIVE': [121599, 37991, 98.99, 3208.69, 10229.47], 'LIZ APP-LIVE': [173671, 65383, 82.31, 3538.31, 9392.92], 'SOZ APP-LIVE': [1093304, 349389, 96.83, 3433.58, 10715.74], 'BLZ APP-EPAPER': [1538991, 91

In [9]:
f = open(filesys_path+file_json_out,"w")
f.write(json.dumps(vars["matrix"]))
f.close()

In [10]:
import pandas as pd

In [11]:
df = pd.read_json(filesys_path+file_json_out)

In [12]:
df.T.sort_index()

,0,1,2,3,4
AAZ APP-EPAPER,4196736.0,218882.0,604.91,1111.55,21400.57
AAZ APP-LIVE,5065888.0,1533188.0,102.32,3541.13,11682.09
AAZ WEB,12583297.0,3313170.0,117.62,2238.95,8404.48
BAT APP-EPAPER,401831.0,18278.0,680.02,1101.80,23786.29
BAT APP-LIVE,126155.0,36839.0,105.97,3324.73,11344.54
BAT WEB,310238.0,81468.0,118.06,2545.20,9634.07
BLZ APP-EPAPER,1538991.0,91816.0,539.80,1126.59,19257.23
BLZ APP-LIVE,1629510.0,542071.0,93.10,3879.39,11641.22
BLZ WEB,93.0,37.0,51.72,400.73,1205.36
BZB APP-EPAPER,4952.0,1336.0,101.89,2365.01,5475.02


In [13]:
df.T.sort_index().to_csv(filesys_path+file_csv_out)